# 쿠팡 '노트북' 검색 후 데이터 스크래핑

In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

## 참고자료

1. get 방식
- https://www.coupang.com/np/search?q=노트북&channel=user&component=&eventCategory= ..... 
- 위와 같이 /np/search? ~~~~ 값이 나옴
- ?뒤 부터 변수와 값을 의미
- 변수와 값은 &로 표시
- 쉽게 페이지를 요청할 수 있음
- 데이터 전송에 제약이 존재하여 큰 데이터는 보낼 수 없음


2. post 방식
- https body에 숨겨서 보내는 방식
- 보안데이터는 post방식으로 숨겨서 전송하는 방식이 안전(개인정보, 회원가입 등)
- 데이터 전송에 제약이 없음

# 1. BeautifulSoup로 스크래핑

In [10]:
url = 'https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page=1&rocketAll=false&searchIndexingToken=1=5&backgroundColor='
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'}
res = requests.get(url, headers = headers)
res.status_code
soup = BeautifulSoup(res.text, 'lxml')

# 2. soup.find_all 함수로 조건에 맞는 값을 items라는 매개변수로 생성

- 여기서 re.compile('^search-product')을 사용한 이유는 광고가 붙지 않은 것은 search-product / 광고가 붙은 것은 search-product search-product__ad-badge이기 때문

In [11]:
items = soup.find_all('li', attrs={'class':re.compile("^search-product")})
items[0].find('div', attrs={'class':'name'}).get_text()

'레노버 2021 ideapad 3D 14, 플래티넘 그레이, Slim3 14IML 81WA00JDKR, 코어i3, 256GB, 4GB, Free DOS'

# 3. items 변수 중 조건에 맞는 값 출력

### 1) 전체 출력

In [12]:
li_name =[]
li_price = []
li_rate =[]
li_rate_count = []
notebook = pd.DataFrame(columns = ['name','price','rate','rate_count'])

for i in items:
    name = i.find('div', attrs={'class':'name'}).get_text()                       # 제품명
    price = i.find('strong', attrs={'class':"price-value"}).get_text()            # 가격
    rate = i.find('em', attrs={'class':"rating"})                                 # 평점
    if rate:                                                                      # em 태그에 텍스트가 없는것(평점이 없음)이 존재
        rate = rate.get_text()                                                    # 따라서 if문으로 분리
    rate_count = i.find('span', attrs={'class':"rating-total-count"})             # 평점 수
    if rate_count:
        rate_count =rate_count.get_text()
    li_name.append(name)
    li_price.append(price)
    li_rate.append(rate)
    li_rate_count.append(rate_count)
            
li_notebook =[li_name, li_price, li_rate, li_rate_count]
for i in range(0,4):
    notebook.iloc[:,i]= li_notebook[i]
    
display(notebook)

,name,price,rate,rate_count
0,"레노버 2021 ideapad 3D 14, 플래티넘 그레이, Slim3 14IML ...","369,000",5.0,(25)
1,"에이수스 2021 X415 14, 투명 실버, 코어i5 10세대, 256GB, 8G...","489,000",5.0,(121)
2,"HP 2021 노트북 14s, 스노우 화이트, 코어i5 11세대, 256GB, 8G...","698,000",4.5,(63)
3,"아이뮤즈 2020 스톰북 14.1, 퓨어 화이트, 셀러론, 64GB, 4GB, WI...","249,000",4.0,(86)
4,이태원클라쓰북 그램스타일 노트북 풀패키지미개봉 NB141LTN41 8세대 14 IP...,"299,000",4.5,(464)
5,HP 엘리트북 840-G4 i5-7300U/8G/SSD512GB/14인치 FHD/윈...,"398,000",4.5,(38)
6,"LG전자 2021 울트라 PC 15.6, 화이트, 코어i3 11세대, 128GB, ...","669,560",5.0,(371)
7,LG전자 2020 울트라 PC 15.6 + 무선마우스 + 마우스패드 + 키스킨 + ...,"674,720",5.0,(1229)
8,삼성 11세대 코어i7 16GB NVMe SSD 1TB 지포스 윈도우10탑재 노트북...,"1,798,000",4.5,(8)
9,"에이수스 2021 X415 14, 슬레이트 그레이, 코어i5 11세대, 256GB,...","589,000",5.0,(77)


### 2) 광고 제외 출력

In [13]:
li_name =[]
li_price = []
li_rate =[]
li_rate_count = []
notebook = pd.DataFrame(columns = ['name','price','rate','rate_count'])

for i in items:
    ad_badge = i.find('span', attrs = {'class' : 'ad-badge-text'})
    if not ad_badge:
        name = i.find('div', attrs={'class':'name'}).get_text()                     
        price = i.find('strong', attrs={'class':"price-value"}).get_text()         
        rate = i.find('em', attrs={'class':"rating"})                              
        if rate:                                                                   
            rate = rate.get_text()                                                   
        rate_count = i.find('span', attrs={'class':"rating-total-count"})             
        if rate_count:
            rate_count =rate_count.get_text()
        li_name.append(name)
        li_price.append(price)
        li_rate.append(rate)
        li_rate_count.append(rate_count)

li_notebook =[li_name, li_price, li_rate, li_rate_count]
for i in range(0,4):
    notebook.iloc[:,i]= li_notebook[i] 

display(notebook)

,name,price,rate,rate_count
0,"레노버 2021 ideapad 3D 14, 플래티넘 그레이, Slim3 14IML ...","369,000",5.0,(25)
1,"에이수스 2021 X415 14, 투명 실버, 코어i5 10세대, 256GB, 8G...","489,000",5.0,(121)
2,"HP 2021 노트북 14s, 스노우 화이트, 코어i5 11세대, 256GB, 8G...","698,000",4.5,(63)
3,"아이뮤즈 2020 스톰북 14.1, 퓨어 화이트, 셀러론, 64GB, 4GB, WI...","249,000",4.0,(86)
4,"LG전자 2021 울트라 PC 15.6, 화이트, 코어i3 11세대, 128GB, ...","669,560",5.0,(371)
5,LG전자 2020 울트라 PC 15.6 + 무선마우스 + 마우스패드 + 키스킨 + ...,"674,720",5.0,(1229)
6,"에이수스 2021 X415 14, 슬레이트 그레이, 코어i5 11세대, 256GB,...","589,000",5.0,(77)
7,"HP 2021 Pavilion 15, 화이트, 코어i5 11세대, 256GB, 8G...","729,680",5.0,(60)
8,"델 2021 Inspiron 15, 플래티넘 실버, 라이젠5 4세대, 256GB, ...","739,000",5.0,(90)
9,"HP 2021 노트북 15s, 내츄럴 실버, 라이젠5 4세대, 256GB, 8GB,...","659,000",5.0,(28)


### 3) 리뷰 100개 이상, 평점 4.5 이상 되는 것만 조회

In [14]:
li_name =[]
li_price = []
li_rate =[]
li_rate_count = []
notebook = pd.DataFrame(columns = ['name','price','rate','rate_count'])

for i in items:
    ad_badge = i.find('span', attrs = {'class' : 'ad-badge-text'})
    if not ad_badge:
        name = i.find('div', attrs={'class':'name'}).get_text()                       
        price = i.find('strong', attrs={'class':"price-value"}).get_text()

        rate = i.find('em', attrs={'class':"rating"})                                
        if rate:                                                                      
            rate = rate.get_text()
    
        rate_count = i.find('span', attrs={'class':"rating-total-count"})
        if rate_count:
            rate_count = rate_count.get_text()
            rate_count = rate_count[1:-1]            # 평점 출력 시 괄호를 제거해주기 위해
        
        # 리뷰 100개 이상, 평점 4.5 이상 출력
        if float(rate) >= 4.5 and int(rate_count) >=100 :
            li_name.append(name)
            li_price.append(price)
            li_rate.append(rate)
            li_rate_count.append(rate_count)
            
li_notebook =[li_name, li_price, li_rate, li_rate_count]
for i in range(0,4):
    notebook.iloc[:,i]= li_notebook[i]
    
display(notebook)

,name,price,rate,rate_count
0,"에이수스 2021 X415 14, 투명 실버, 코어i5 10세대, 256GB, 8G...","489,000",5.0,121
1,"LG전자 2021 울트라 PC 15.6, 화이트, 코어i3 11세대, 128GB, ...","669,560",5.0,371
2,LG전자 2020 울트라 PC 15.6 + 무선마우스 + 마우스패드 + 키스킨 + ...,"674,720",5.0,1229
3,"에이수스 2021 Vivobook 15, 드리미화이트, 코어i5 11세대, 512G...","668,800",4.5,102
4,"LG전자 2020 울트라 PC 15.6, 화이트, 펜티엄, 256GB, 4GB, W...","599,000",5.0,212
5,"Apple 2020 맥북 에어 13, 스페이스 그레이, M1, 256GB, 8GB,...","1,135,000",5.0,6571
6,"레노버 2021 IdeaPad Slim3 15.6, 골드, 펜티엄, 128GB, 4...","416,760",5.0,134
7,"삼성전자 2020 갤럭시북 S 13.3, 머큐리 그레이, 코어i5, 256GB, 8...","859,000",5.0,704
8,"레노버 2020 IdeaPad Slim1 14, 블루, 셀러론, 64GB, 4GB,...","450,000",4.5,226
9,"삼성전자 2020 노트북 15.6, 퓨어 화이트, 코어i5 10세대, 256GB, ...","1,535,770",5.0,2657


### 4) 1~5p에서 광고가 아니며, 리뷰 100개 이상, 평점 4.5 이상 되는 것만 조회

In [15]:
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'}
li_page = []
li_name =[]
li_price = []
li_rate =[]
li_rate_count = []
li_link = []
notebook = pd.DataFrame(columns = ['page','name','price','rate','rate_count','link'])

for i in range(1,6):
    url = 'https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page={}&rocketAll=false&searchIndexingToken=1=5&backgroundColor='.format(i)
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text, 'lxml')
    items = soup.find_all('li', attrs = {'class': re.compile('^search-product')})

    for item in items:
        ad_badge = item.find('span', attrs = {'class' : 'ad-badge-text'})
        if not ad_badge:
            page = i
            name = item.find('div', attrs = {'class' : 'name'}).get_text()
            price = item.find('strong', attrs = {'class' : 'price-value'}).get_text()
            
            rate = item.find('em', attrs = {'class' : 'rating'})
            if rate:
                rate = rate.get_text() 
                
            ## 위의 rate에서 else: continue를 사용했다면, 아래의 내용은 위 rate에 맞춰 들여쓰기를 할 필요가 없다!!
            
            
                rate_count = item.find('span', attrs = {'class' : 'rating-total-count'})
                if rate_count:
                    rate_count = rate_count.get_text()[1:-1]
                link = 'https://www.coupang.com/' + item.find('a', attrs = {'class' : 'search-product-link'})['href']
                if float(rate) >= 4.5 and int(rate_count) >= 100:
                    li_page.append(page)
                    li_name.append(name)
                    li_price.append(price)
                    li_rate.append(rate)
                    li_rate_count.append(rate_count)
                    li_link.append(link)

li_notebook =[li_page,li_name, li_price, li_rate, li_rate_count, li_link]
for i in range(0,6):
    notebook.iloc[:,i] = li_notebook[i]
        
display(notebook)

,page,name,price,rate,rate_count,link
0,1,"에이수스 2021 X415 14, 투명 실버, 코어i5 10세대, 256GB, 8G...","489,000",5.0,121,https://www.coupang.com//vp/products/492329013...
1,1,"LG전자 2021 울트라 PC 15.6, 화이트, 코어i3 11세대, 128GB, ...","669,560",5.0,371,https://www.coupang.com//vp/products/467998925...
2,1,LG전자 2020 울트라 PC 15.6 + 무선마우스 + 마우스패드 + 키스킨 + ...,"674,720",5.0,1229,https://www.coupang.com//vp/products/491910491...
3,1,"에이수스 2021 Vivobook 15, 드리미화이트, 코어i5 11세대, 512G...","668,800",4.5,102,https://www.coupang.com//vp/products/496277770...
4,1,"LG전자 2020 울트라 PC 15.6, 화이트, 펜티엄, 256GB, 4GB, W...","599,000",5.0,212,https://www.coupang.com//vp/products/160532803...
5,1,"Apple 2020 맥북 에어 13, 스페이스 그레이, M1, 256GB, 8GB,...","1,135,000",5.0,6571,https://www.coupang.com//vp/products/432248113...
6,1,"레노버 2021 IdeaPad Slim3 15.6, 골드, 펜티엄, 128GB, 4...","416,760",5.0,134,https://www.coupang.com//vp/products/560743789...
7,1,"삼성전자 2020 갤럭시북 S 13.3, 머큐리 그레이, 코어i5, 256GB, 8...","859,000",5.0,704,https://www.coupang.com//vp/products/231624269...
8,1,"레노버 2020 IdeaPad Slim1 14, 블루, 셀러론, 64GB, 4GB,...","450,000",4.5,226,https://www.coupang.com//vp/products/223867010...
9,1,"삼성전자 2020 노트북 15.6, 퓨어 화이트, 코어i5 10세대, 256GB, ...","1,535,770",5.0,2657,https://www.coupang.com//vp/products/469831855...


### 5) 엑셀파일로 저장

In [336]:
notebook.to_excel('C:/Users/Park/Desktop/data/sample.xlsx')